In [1]:
#!fsharp
#load "../RayTracerChallenge/RayTracerChallenge/scripts/load.fsx"

open RayTracerChallenge.Utilities
open RayTracerChallenge.Tuples
open RayTracerChallenge.Color
open RayTracerChallenge.Canvas
open RayTracerChallenge.PPM
open RayTracerChallenge.Ray

In [1]:
#!fsharp
let rayOrigin = point(0.0, 0.0, -5.0)
let wallZ = 10.0
let wallSize = 7.0
let canvasSize = 200.0
let pixelSize = wallSize / canvasSize
let halfSize = wallSize / 2.0
let c = canvas (roundToInt canvasSize, roundToInt canvasSize)

let compute x y =
    let worldX = halfSize - pixelSize * y
    let worldY = -halfSize + pixelSize * x
    let position = point(worldX, worldY, wallZ)
    let r = ray rayOrigin (normalize (position - rayOrigin))
    let xs = intersect Sphere None r
    match hit xs with
    | Some _ -> color(0.5, 0.5, 1.0)
    | None   -> color(0.0, 0.0, 0.0)

let newC = mapxy (fun x y _ -> compute (float x) (float y)) c

let ppmString = canvasToPPM newC

writeToPPM newC (System.IO.Path.Combine(__SOURCE_DIRECTORY__, "../RayTracerChallenge/Images/circle.ppm"))

In [1]:
#!fsharp
#r "nuget: SkiaSharp"

open SkiaSharp
open System.IO

In [1]:
#!fsharp
let bitmap = new SKBitmap(200, 200)
let convert (c : RayTracerChallenge.Color.Color) = SKColor(byte (255.*c.Red), byte (255.*c.Green), byte (255.*c.Blue), byte 255)
bitmap
iterxy (fun x y c -> bitmap.SetPixel(x, y, convert c)) newC
bitmap
let image = SKImage.FromBitmap(bitmap)
let data = image.Encode()
let file = File.OpenWrite(@"../RayTracerChallenge/Images/circle.png")
data.SaveTo(file)
file.Close()
file.Dispose()